In [3]:
import os
import logging
import numpy as np
import tensorflow as tf
from types import SimpleNamespace

import utils
from data_handler import DataHandler
from genetic_optimizer import ChromosomeHelper, GeneticAlgorithm
from trainer import retrain_and_evaluate_best_model

def build_derived_ga_params(config):
    """
    Dynamically builds the VARTYPE and VARBOUND lists for a unified chromosome
    that contains genes for all possible streams.
    """
    logger = logging.getLogger()
    logger.info("Dynamically building unified GA parameters from dictionary config...")

    ns = config.network_structure


    CONV_GENE_ORDER = ['is_active', 'filters', 'kernel_size', 'activation', 'padding', 'strides', 'use_bn', 'use_pool', 'pool_type']
    POST_CONV_GENE_ORDER = ['global_pooling', 'use_dropout', 'dropout_rate']
    LSTM_GENE_ORDER = ['is_active', 'units', 'activation', 'recurrent_dropout', 'use_bn', 'use_dropout', 'dropout_rate']
    MLP_GENE_ORDER = ['is_active', 'use_bn', 'use_dense', 'units', 'activation', 'use_dropout', 'dropout_rate']
    NETWORK_PARAMS_GENE_ORDER = ['l1_reg', 'learning_rate']
    VARTYPE_MAP = {'dropout_rate': 'real', 'recurrent_dropout': 'real', 'l1_reg': 'real', 'learning_rate': 'real'}


    varbound_conv_block = [getattr(ns.conv_block_bounds, key) for key in CONV_GENE_ORDER]
    vartype_conv_block = [VARTYPE_MAP.get(key, 'int') for key in CONV_GENE_ORDER]
    varbound_post_conv = [getattr(ns.post_conv_bounds, key) for key in POST_CONV_GENE_ORDER]
    vartype_post_conv = [VARTYPE_MAP.get(key, 'int') for key in POST_CONV_GENE_ORDER]
    varbound_lstm_block = [getattr(ns.lstm_block_bounds, key) for key in LSTM_GENE_ORDER]
    vartype_lstm_block = [VARTYPE_MAP.get(key, 'int') for key in LSTM_GENE_ORDER]
    vartype_mlp_block = [VARTYPE_MAP.get(key, 'int') for key in MLP_GENE_ORDER]
    varbound_mlp_block = [getattr(ns.mlp_block_bounds, key) for key in MLP_GENE_ORDER]
    varbound_network_params = [getattr(ns.network_params_bounds, key) for key in NETWORK_PARAMS_GENE_ORDER]
    vartype_network_params = [VARTYPE_MAP.get(key, 'int') for key in NETWORK_PARAMS_GENE_ORDER]


    config.derived_ga_params.VARBOUND_CONV1D = varbound_conv_block * ns.N_CONV_BLOCKS_1D + varbound_post_conv
    config.derived_ga_params.VARTYPE_CONV1D = vartype_conv_block * ns.N_CONV_BLOCKS_1D + vartype_post_conv
    config.derived_ga_params.VARBOUND_CONV2D = varbound_conv_block * ns.N_CONV_BLOCKS_2D + varbound_post_conv
    config.derived_ga_params.VARTYPE_CONV2D = vartype_conv_block * ns.N_CONV_BLOCKS_2D + vartype_post_conv
    config.derived_ga_params.VARBOUND_HP_LSTM = varbound_lstm_block * ns.N_LSTM_BLOCKS
    config.derived_ga_params.VARTYPE_HP_LSTM = vartype_lstm_block * ns.N_LSTM_BLOCKS
    config.derived_ga_params.VARBOUND_MLP_SHARED = varbound_mlp_block * ns.N_MLP_BLOCKS + varbound_network_params
    config.derived_ga_params.VARTYPE_MLP_SHARED = vartype_mlp_block * ns.N_MLP_BLOCKS + vartype_network_params

    logger.info("Unified GA parameters built successfully.")
    return config
    
def get_paper_config(dataname="AREM", subjects_train=None, subjects_val=None, subjects_test=None):
    """
    Sets up the configuration parameters exactly as described in the paper.
    """
    config = SimpleNamespace(
        general=SimpleNamespace(
            DATANAME_PREFIX=dataname,
            BASE_CSV_PATH=f"data/{dataname}.csv",
            RESULTS_DIR="results",
            FS=20,
            TRAIN_SUBJECTS=subjects_train or ['A1','A2','A3',],
            VALID_SUBJECTS=subjects_val or ['A4','A5'],
            TEST_SUBJECTS=subjects_test or ['A6']
        ),
        data_preprocessing=SimpleNamespace(
            WINDOW_SIZE_SECONDS_OPTIONS=[0.25, 0.5],
            OVERLAP_RATIO_OPTIONS=[0.25],
            DATA_CACHE_DIR="cache_data",
            INITIAL_NPERSEG_DIVISOR=5
        ),
        ga=SimpleNamespace(
            POPULATION_SIZE=50,
            GENERATIONS=30,
            MAX_CHILDREN_PER_GENERATION_FACTOR=0.4,
            CROSSOVER_PROBABILITY=0.8,
            MUTATION_PROBABILITY=0.2,
            OPTIMIZATION_METRIC="f1_macro",
            N_TRIALS_PER_EVALUATION=1,
            EPOCHS_PER_TRIAL=10,
            FITNESS_VERBOSE_LEVEL=1,
            PATIENCE_EARLY_STOPPING=5,
            BATCH_SIZES_OPTIONS=[64, 128, 256, 512, 1024],
            ACTIVE_STREAMS=SimpleNamespace(
                cnn_1d=True,  # Time-domain stream
                cnn_2d=True,  # Frequency-domain stream (STFT)
                lstm=False    # Paper focused on CNN dual-stream
            ),
            DEV_RATIO=0.2 # Ratio of training used for internal GA evaluation
        ),
        network_structure=SimpleNamespace(
            N_CONV_BLOCKS_1D=2,
            N_CONV_BLOCKS_2D=2,
            N_LSTM_BLOCKS=0,
            N_MLP_BLOCKS=2,
            # Search space bounds for CNN layers (Paper lines 107-111)
            conv_block_bounds=SimpleNamespace(
                is_active=[0, 1],
                filters=[1, 500],
                kernel_size=[1, 8],
                activation=[0, 2], # ReLU, Tanh, Sigmoid
                padding=[0, 1],    # Valid, Same
                strides=[1, 2],
                use_bn=[0, 1],
                use_pool=[0, 1],
                pool_type=[0, 1]   # Max, Average
            ),
            post_conv_bounds=SimpleNamespace(
                global_pooling=[0, 1],
                use_dropout=[0, 1],
                dropout_rate=[0.4, 0.5]
            ),
            lstm_block_bounds=SimpleNamespace(
                is_active=[0, 1],
                units=[32, 256],
                activation=[0, 2],
                recurrent_dropout=[0.0, 0.5],
                use_bn=[0, 1],
                use_dropout=[0, 1],
                dropout_rate=[0.1, 0.5]
            ),
            mlp_block_bounds=SimpleNamespace(
                is_active=[0, 1],
                use_bn=[0, 1],
                use_dense=[1, 1],
                units=[3, 1024],
                activation=[0, 2],
                use_dropout=[0, 1],
                dropout_rate=[0.4, 0.5]
            ),
            network_params_bounds=SimpleNamespace(
                l1_reg=[1e-6, 1e-2],
                learning_rate=[1e-6, 1e-2]
            )
        ),
        loss_function=SimpleNamespace(
            GAMMA_OPTIONS=[1.0, 2.0, 3.0, 5.0],
            CLASS_WEIGHT_MULTIPLIER_OPTIONS=[1, 2, 5, 10]
        ),
        mappings=SimpleNamespace(
            ACTIVATION_MAP=["relu", "tanh", "sigmoid"],
            PADDING_MAP=["valid", "same"]
        ),
        training=SimpleNamespace(TRAINING_TIMEOUT_SECONDS=300),
        derived_ga_params=SimpleNamespace() # Populated at runtime
    )
    return config

def main():
    # 1. Setup Environment
    config = get_paper_config()
    run_dir = os.path.join(config.general.RESULTS_DIR, f"paper_run_{utils.get_timestamp()}")
    logger = utils.setup_logger(run_dir, config.general.DATANAME_PREFIX)
    
    logger.info("Starting Automated Dual-Stream Network Design...")

    # 2. Data Preparation
    # Note: DataHandler must implement subject-based splitting as per paper Section 2.2
    data_handler = DataHandler(config.general.BASE_CSV_PATH, config, logger)
    data_handler.load_data()
    data_handler.preprocess_labels()
    
    # Pre-generate sliding window datasets for the GA search space
    data_handler.generate_and_cache_datasets(
        window_size_options=config.data_preprocessing.WINDOW_SIZE_SECONDS_OPTIONS,
        overlap_ratio_options=config.data_preprocessing.OVERLAP_RATIO_OPTIONS,
        fs=config.general.FS,
        cache_dir=config.data_preprocessing.DATA_CACHE_DIR,
        train_subjects=config.general.TRAIN_SUBJECTS,
        valid_subjects=config.general.VALID_SUBJECTS,
        test_subjects=config.general.TEST_SUBJECTS
    )

    # 3. Setup Genetic Algorithm
    config = build_derived_ga_params(config)
    
    chromosome_helper = ChromosomeHelper(data_handler.num_classes, config, logger)
    
    ga = GeneticAlgorithm(
        chromosome_helper=chromosome_helper,
        config=config,
        logger=logger,
        run_dir=run_dir
    )

    # 4. Execute Neural Architecture Search (NAS)
    best_individual, is_finished = ga.run()

    # 5. Final Evaluation (Retrain on Train+Val, test on Test)
    if is_finished:
        logger.info("NAS complete. Retraining best model for final evaluation...")
        retrain_and_evaluate_best_model(
            best_individual_parts=best_individual[0],
            best_individual_fitness=best_individual[1],
            config=config,
            logger=logger,
            run_dir=run_dir,
            epochs_retrain=50 # Final convergence
        )


In [4]:
main()

2026-02-08 07:41:05,582 - AREM - utils.py - INFO - TensorFlow version: 2.18.0
2026-02-08 07:41:05,582 - AREM - utils.py - INFO - TensorFlow version: 2.18.0
2026-02-08 07:41:05,585 - AREM - utils.py - INFO - Available GPUs: []
2026-02-08 07:41:05,585 - AREM - utils.py - INFO - Available GPUs: []
2026-02-08 07:41:05,586 - AREM - utils.py - WARNING - No GPU detected by TensorFlow. Running on CPU will be slow.
2026-02-08 07:41:05,586 - AREM - utils.py - WARNING - No GPU detected by TensorFlow. Running on CPU will be slow.
2026-02-08 07:41:05,587 - AREM - 2658416851.py - INFO - Starting Automated Dual-Stream Network Design...
2026-02-08 07:41:05,587 - AREM - 2658416851.py - INFO - Starting Automated Dual-Stream Network Design...
2026-02-08 07:41:05,588 - AREM - data_handler.py - INFO - Loading data from data/AREM.csv
2026-02-08 07:41:05,588 - AREM - data_handler.py - INFO - Loading data from data/AREM.csv
2026-02-08 07:41:05,643 - AREM - data_handler.py - INFO - Data loaded successfully. Sh

Epoch 1/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.3833 - f1_score_macro: 0.3500 - loss: 8.8278 - val_accuracy: 0.1418 - val_f1_score_macro: 0.0355 - val_loss: 16.6969
Epoch 2/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6544 - f1_score_macro: 0.6178 - loss: 3.1114 - val_accuracy: 0.1710 - val_f1_score_macro: 0.0740 - val_loss: 22.2536
Epoch 3/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7416 - f1_score_macro: 0.7390 - loss: 2.4284 - val_accuracy: 0.2759 - val_f1_score_macro: 0.1857 - val_loss: 20.6690
Epoch 4/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7985 - f1_score_macro: 0.7807 - loss: 1.5251 - val_accuracy: 0.2801 - val_f1_score_macro: 0.1789 - val_loss: 21.9553
Epoch 5/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7518 - f1_score_macro: 0.7278 - loss: 1.5479 - val_accuracy: 0.2503 - val_f1_score_macro: 0.1565 - val_loss: 23.4854
Epoch 6/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8666 - f1_score_macro: 0

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fa30c19cc70>>
Traceback (most recent call last):
  File "/home/conficker/anaconda3/envs/tf218/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fa2820463b0>
Traceback (most recent call last):
  File "/home/conficker/anaconda3/envs/tf218/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 

KeyboardInterrupt

